In [1]:
!pip install "numpy<2"
!pip install "sentence-transformers==2.2.2"
!pip install "InstructorEmbedding>=1.0.1"
!pip install "langchain>=0.2.0"
!pip install "langchain-community>=0.0.11"
!pip install "torch>=2.0"
!pip install -U "sentence-transformers==2.2.2"
!pip install -U "InstructorEmbedding>=1.0.1"
!pip install sentence-transformers



  Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 4.1.0
    Uninstalling sentence-transformers-4.1.0:
      Successfully uninstalled sentence-transformers-4.1.0
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [2]:
import pandas as pd 
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt 
import seaborn as sns
from langchain_google_genai import ChatGoogleGenerativeAI 
from langchain.prompts import PromptTemplate

import re

In [3]:
import pprint
import google.generativeai as palm

/Users/phulusoravhuhali/Documents/phuluso_all/phuluso_data_science/Data_science/mine/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

load_dotenv()

True

In [5]:
api_key = os.environ.get("api_key")


In [6]:
palm.configure(api_key=api_key)

In [7]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


In [8]:

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key,
    temperature=0.9,

)


In [9]:

prompt = "can i give you a df and you give me a query "
response = llm.invoke(prompt)
print(response.content) 



Yes, you can provide me with a DataFrame (you'll need to represent it in a way I can understand, such as a CSV string, a list of lists, or a dictionary), and I will try my best to give you a corresponding SQL query (or a Pandas/Python query if you specify).  Please tell me:

* **What is the DataFrame representing?** (e.g., customer data, sales transactions, website visits)  This helps me understand the context and write a more meaningful query.
* **What format are you providing the DataFrame in?** (e.g., CSV, list of lists, dictionary)  Please provide the data in a clear and well-structured way.
* **What is the desired output of the query?** (e.g., "average sales per month", "list of customers who made purchases over $100", "number of unique visitors per day") Be as specific as possible.
* **What database system are you targeting?** (e.g., MySQL, PostgreSQL, SQLite, or just Pandas) This affects the syntax of the query.  If you're just using Pandas, say so!

The more information you giv

In [10]:
dc  = pd.read_csv("customer_support_tickets.csv")
dc['extracted_question'] = dc['Ticket Description'].str.extract(r'(\?.*$)')
dc['cleaned_question'] = dc['extracted_question'].str.lower().str.strip()
dc = dc.dropna(subset=['cleaned_question'])


In [11]:
text = " ".join(dc['extracted_question'] .astype(str).tolist())
text = re.sub(r"[^A-Za-z\s]", "", text)
text = text.lower()
text

'   report feedback customer service is your best im using the original charger that came with my productpurchased but its not charging properly   ive noticed that the issue occurs consistently when i use a specific feature or application on my productpurchased  i need assistance as soon as possible because its affecting my work and productivity   i can reset my password by entering the following my password still valid password is expired ive recently updated the firmware of my productpurchased and the issue started happening afterward could it be related to the update     please contact me im unable to find the option to perform the desired action in the productpurchased could you please guide me through the steps  visit httpsforumejconop ive reviewed the troubleshooting steps on the official support website but they didnt resolve the problem  for those who live in the us you can use the product im unable to find the option to perform the desired action in the productpurchased could 

In [12]:
prompt = f"in a column called Ticket Description in {text} can you give me 5 most asked quesions just give me 1 to 5  simple answers, keep it simple  "
response = llm.invoke(prompt)
print(response.content) 

Here's a categorization of the customer feedback from the "Ticket Description" column, simplified into 5 frequently asked questions (FAQs) with simple answers:

**1. My product isn't charging properly.**

* **Answer:** Try a different charging cable or outlet. If the problem persists, contact customer support for troubleshooting or repair.

**2.  I'm experiencing unexpected glitches/intermittent problems.**

* **Answer:**  Try restarting your device. If the issue continues, check for software updates or contact customer support.  They may need to investigate further.

**3. I can't find a specific feature or option.**

* **Answer:** Consult your user manual or the product's official website for instructions. If you still can't find it, contact customer support.

**4. A recent update caused problems.**

* **Answer:** Contact customer support to report the issue and provide details about the update and the problems it caused. They may have a fix or workaround.

**5. I'm concerned about th

# Using langchain

In [13]:
from langchain.document_loaders.csv_loader import CSVLoader

In [14]:
load = CSVLoader(file_path="customer_support_tickets.csv",source_column="Ticket Description")
df  = load.load()

### testing embedings 

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
# Initialize with a simple model like "all-MiniLM-L6-v2" (very popular for sentence embeddings)
emb = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

w = emb.embed_query("what is today's date")



# creating a vector database

In [17]:
from langchain.vectorstores import FAISS
#create vector databases
vdb = FAISS.from_documents(documents=df,embedding=emb)

#### for things already in the database : finding similer questiong to the one you asked

In [18]:
#retrieve vecot databases 
pull_vectore =vdb.as_retriever()

In [19]:
questio = pull_vectore.get_relevant_documents("my charger is not working properly")
questio

/var/folders/hv/m7d19hld69qb83g8n_7x10tc0000gn/T/ipykernel_5628/1956359734.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  questio = pull_vectore.get_relevant_documents("my charger is not working properly")


[Document(id='7e8f88c0-f60f-47a4-8300-360c0443aa0a', metadata={'source': 'My {product_purchased} is making strange noises and not functioning properly. I suspect there might be a hardware issue. Can you please help me with this?\n\nMy {product_p word} is being "unresponsive", I I\'m using the original charger that came with my {product_purchased}, but it\'s not charging properly.', 'row': 3530}, page_content='Ticket ID: 3531\nCustomer Name: Peter Anderson\nCustomer Email: matthew14@example.com\nCustomer Age: 32\nCustomer Gender: Other\nProduct Purchased: Adobe Photoshop\nDate of Purchase: 2021-05-22\nTicket Type: Technical issue\nTicket Subject: Product compatibility\nTicket Description: My {product_purchased} is making strange noises and not functioning properly. I suspect there might be a hardware issue. Can you please help me with this?\n\nMy {product_p word} is being "unresponsive", I I\'m using the original charger that came with my {product_purchased}, but it\'s not charging prop

## Asking the AI from the data in the DB

In [20]:
prompt_temp = """
Answer the question based on the context and Question, dont come up with your own answers , 
say i dont know if there is no answer in the here.
context : {context}
QUESTION : {question}
"""
promptt = PromptTemplate(
    template=prompt_temp,input_variables=['context','question']
)

In [21]:
from langchain.chains import RetrievalQA

chain  = RetrievalQA.from_chain_type(llm = llm,
    retriever =  pull_vectore,
    input_key = "query",
    return_source_documents=True,
    chain_type_kwargs = {'prompt' :promptt}
    )

In [22]:
chain("how munch money did you make last year ")

/var/folders/hv/m7d19hld69qb83g8n_7x10tc0000gn/T/ipykernel_5628/2878782360.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain("how munch money did you make last year ")


{'query': 'how munch money did you make last year ',
 'result': "I don't know.  The provided text contains customer service tickets and doesn't include information about revenue or profits.",
 'source_documents': [Document(id='4c327fc4-47bf-4254-a644-9bdd6fff302e', metadata={'source': "I'm having an issue with the {product_purchased}. Please assist.\n\nI want to see your product\n\nA product bought of $0 in the 1st quarter of 2014. It's my phone. I want it. I've checked for any available software updates for my {product_purchased}, but there are none.", 'row': 6990}, page_content="Ticket ID: 6991\nCustomer Name: Ricky Herrera\nCustomer Email: ubooth@example.org\nCustomer Age: 66\nCustomer Gender: Male\nProduct Purchased: Nintendo Switch Pro Controller\nDate of Purchase: 2020-10-13\nTicket Type: Product inquiry\nTicket Subject: Product recommendation\nTicket Description: I'm having an issue with the {product_purchased}. Please assist.\n\nI want to see your product\n\nA product bought of